In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  
import datetime as dt

/Users/fqian/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_predict
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import PolynomialFeatures

In [36]:
responses = pd.read_csv('Responses Full 0328.csv')
responses.head()

,Unnamed: 0,actions,bucket_name,discount_asked,id,nightly_price,nightly_price_scraped,response,availability,price,discount
0,0,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,8823897,55,$60,"Hi Reuben, my name is Michele and I am interes...",-1,NaN,NaN
1,1,[],days1_weeks1,0.7,7803063,65,$120,No can do. Cleaning fee alone is $50. *******...,0,NaN,NaN
2,2,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.8,8860745,92,$112,I arrive late* (11pm) *************** Hi Miche...,-1,NaN,NaN
3,3,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,6038278,32,$35,Hi the room is available but the price is firm...,0,NaN,NaN
4,4,[],days1_weeks1,0.7,6400170,74,$130,"Hi Andres, my name is Michele and I am interes...",-1,NaN,NaN


#### Construct 11 variables: 3 types of orphans, 2 types of advances, and 3x2=6 types of interactions

In [37]:
opening_attr = responses["bucket_name"].values
# orphan = []
# advance = []
N = len(opening_attr)
orp_1 = np.zeros(N)
orp_2 = np.zeros(N)
orp_3 = np.zeros(N)
adv_1 = np.zeros(N)
adv_2 = np.zeros(N)

for (i,x) in enumerate(opening_attr):
    if x == "days1_weeks1":
        orp_1[i] = 1
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 1
        adv_2[i] = 0
    elif x == "days1_weeks2":
        orp_1[i] = 1
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 1
    elif x == "days1_weeksM":
        orp_1[i] = 1
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 0
    elif x == "days2_weeks1":
        orp_1[i] = 0
        orp_2[i] = 1
        orp_3[i] = 0
        adv_1[i] = 1
        adv_2[i] = 0
    elif x == "days2_weeks2":
        orp_1[i] = 0
        orp_2[i] = 1
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 1
    elif x == "days2_weeksM":
        orp_1[i] = 0
        orp_2[i] = 1
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 0
    elif x == "days3_weeks1":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 1
        adv_1[i] = 1
        adv_2[i] = 0
    elif x == "days3_weeks2":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 1
        adv_1[i] = 0
        adv_2[i] = 1
    elif x == "days3_weeksM":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 1
        adv_1[i] = 0
        adv_2[i] = 0
    elif x == "daysM_weeks1":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 1
        adv_2[i] = 0
    elif x == "daysM_weeks2":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 1
    elif x == "daysM_weeksM":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 0
        
responses["orp_1"]=orp_1
responses["orp_2"]=orp_2
responses["orp_3"]=orp_3
responses["adv_1"]=adv_1
responses["adv_2"]=adv_2

In [38]:
responses["orp_1, adv_1"] = responses["orp_1"]*responses["adv_1"]
responses["orp_1, adv_2"] = responses["orp_1"]*responses["adv_2"]
responses["orp_2, adv_1"] = responses["orp_2"]*responses["adv_1"]
responses["orp_2, adv_2"] = responses["orp_2"]*responses["adv_2"]
responses["orp_3, adv_1"] = responses["orp_3"]*responses["adv_1"]
responses["orp_3, adv_2"] = responses["orp_3"]*responses["adv_2"]

In [41]:
responses.head()

,Unnamed: 0,actions,bucket_name,discount_asked,id,nightly_price,nightly_price_scraped,response,availability,price,...,orp_2,orp_3,adv_1,adv_2,"orp_1, adv_1","orp_1, adv_2","orp_2, adv_1","orp_2, adv_2","orp_3, adv_1","orp_3, adv_2"
0,0,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,8823897,55,$60,"Hi Reuben, my name is Michele and I am interes...",-1,NaN,...,0,0,1,0,1,0,0,0,0,0
1,1,[],days1_weeks1,0.7,7803063,65,$120,No can do. Cleaning fee alone is $50. *******...,0,NaN,...,0,0,1,0,1,0,0,0,0,0
2,2,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.8,8860745,92,$112,I arrive late* (11pm) *************** Hi Miche...,-1,NaN,...,0,0,1,0,1,0,0,0,0,0
3,3,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,6038278,32,$35,Hi the room is available but the price is firm...,0,NaN,...,0,0,1,0,1,0,0,0,0,0
4,4,[],days1_weeks1,0.7,6400170,74,$130,"Hi Andres, my name is Michele and I am interes...",-1,NaN,...,0,0,1,0,1,0,0,0,0,0


In [43]:
inside_airbnb = pd.read_csv('listings_nyc.csv')

shared = inside_airbnb["room_type"].values
inside_airbnb["shared"] = [x=="Private room" for x in shared]
# shared_binary

instant = inside_airbnb["instant_bookable"].values
inside_airbnb["instant"]  = [x=="t" for x in shared]
# instant_binary

In [44]:
available= pd.merge(responses[responses["availability"]!=-1], inside_airbnb, how='inner',left_on=['id'], right_on=['id'])

In [71]:
sample_count = available.groupby("bucket_name")["id"].count()
print len(available)
print sample_count

104
bucket_name
days1_weeks1    10
days1_weeksM     9
days2_weeks1     9
days2_weeks2    12
days2_weeksM    10
days3_weeks1    11
days3_weeks2     5
days3_weeksM     9
daysM_weeks1    10
daysM_weeks2     9
daysM_weeksM    10
Name: id, dtype: int64


In [45]:
cal_param = ["orp_1", "orp_2", "orp_3", "adv_1", "adv_2", "orp_1, adv_1", "orp_1, adv_2", "orp_2, adv_1", "orp_2, adv_2", "orp_3, adv_1", "orp_3, adv_2"]
all_param = cal_param + ["discount_asked", "nightly_price", "shared", "instant"]

In [49]:
X_train = available[all_param].values
Y_train = available["availability"].values

In [85]:
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)
# precisions=cross_validation.cross_val_score(lr, X, y, cv=kf_total, scoring='precision')

lr_scores = -cross_val_score(logistic, X_train, Y_train, scoring='mean_squared_error', cv=5)

#### Coefficient

In [93]:
zip(all_param, np.ravel(logistic.coef_))

[('orp_1', -0.67978310106545903),
 ('orp_2', 0.44899177569948689),
 ('orp_3', 0.48967858479423343),
 ('adv_1', -0.50649697324094611),
 ('adv_2', -0.45148027073001312),
 ('orp_1, adv_1', 0.2947336720609543),
 ('orp_1, adv_2', 0.0),
 ('orp_2, adv_1', -0.069988895069940132),
 ('orp_2, adv_2', 0.43221886011959287),
 ('orp_3, adv_1', -0.04378944953693914),
 ('orp_3, adv_2', 0.41504606143040967),
 ('discount_asked', -0.24696961430142622),
 ('nightly_price', 0.0044412580210620861),
 ('shared', 0.036177417352815128),
 ('instant', 0.0)]

#### Accuracy

In [87]:
logistic.score(X_train, Y_train)

0.69230769230769229

In [88]:
lr_scores

array([ 0.36363636,  0.72727273,  0.55      ,  0.4       ,  0.3       ])

In [39]:
df_analysis.head()

,Unnamed: 0,actions,bucket_name,discount_asked,id,nightly_price,nightly_price_scraped,response,availability,price_x,...,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,shared_space,instant,orphan,advance
0,0,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,8823897,55,$60,"Hi Reuben, my name is Michele and I am interes...",-1,NaN,...,f,flexible,f,f,3,2.92,True,False,1,1
1,1,[],days1_weeks1,0.7,7803063,65,$120,No can do. Cleaning fee alone is $50. *******...,0,NaN,...,f,strict,t,t,3,1.01,True,False,1,1
2,2,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.8,8860745,92,$112,I arrive late* (11pm) *************** Hi Miche...,-1,NaN,...,f,moderate,f,f,1,0.69,False,False,1,1
3,3,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,6038278,32,$35,Hi the room is available but the price is firm...,0,NaN,...,t,strict,f,f,8,4.44,True,False,1,1
4,4,[],days1_weeks1,0.7,6400170,74,$130,"Hi Andres, my name is Michele and I am interes...",-1,NaN,...,t,strict,f,f,1,0.64,False,False,1,1


In [42]:
df_regress = df_analysis[df_analysis["availability"]>=0][["bucket_name", "discount_asked", "availability", "price_x", 
                         "discount", "host_listings_count", "review_scores_rating", "instant", "shared_space", "orphan","advance"]]

In [44]:
print len(df_regress)
df_regress.head()

104


,bucket_name,discount_asked,availability,price_x,discount,host_listings_count,review_scores_rating,instant,shared_space,orphan,advance
1,days1_weeks1,0.7,0,NaN,NaN,3,96,False,True,1,1
3,days1_weeks1,0.6,0,NaN,NaN,8,89,False,True,1,1
9,days1_weeks1,0.8,0,NaN,NaN,1,100,False,True,1,1
10,days1_weeks1,0.9,0,NaN,NaN,6,NaN,False,True,1,1
12,days1_weeks1,0.6,0,NaN,NaN,6,96,False,True,1,1


In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_predict

In [51]:
from sklearn import cross_validation, linear_model

In [47]:
X_train = df_regress[["host_listings_count", "review_scores_rating", "instant", "shared_space", "orphan", "advance"]].values
Y_train = df_regress["availability"]

In [52]:
LR = LogisticRegression()
LR_scores = (-cross_validation.cross_val_score(LR, X_train, Y_train, 
                                          scoring='mean_squared_error', cv=10))**(0.5)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').